# 12.1

## (c)

In [1]:
import numpy as np

In [2]:
from bayesian.bbn import build_bbn
from bayesian.utils import make_key

def f_burglary(burglary):
    if burglary:
        return 0.01
    else:
        return 0.99

def f_earthquake(earthquake):
    if earthquake:
        return 10e-6
    else:
        return 1.-10e-6

def f_alarm(burglary, earthquake, alarm):
    table = dict()
    table['fff'] = 0.999
    table['fft'] = 0.001
    table['ftf'] = 0.59
    table['ftt'] = 0.41
    table['tff'] = 0.05
    table['tft'] = 0.95
    table['ttf'] = 0.02
    table['ttt'] = 0.98
    return table[make_key(burglary, earthquake, alarm)]

def f_radio(earthquake, radio):
    table = dict()
    table['ff'] = 1.0
    table['ft'] = 0.0
    table['tf'] = 0.0
    table['tt'] = 1.0
    return table[make_key(earthquake, radio)]


g = build_bbn(
    f_burglary,
    f_earthquake,
    f_alarm,
    f_radio)
g.q()
g.q(alarm=1,radio=1)
g.q(alarm=1)

+------------+-------+----------+
| Node       | Value | Marginal |
+------------+-------+----------+
| alarm      | False | 0.989506 |
| alarm      | True  | 0.010494 |
| burglary   | False | 0.990000 |
| burglary   | True  | 0.010000 |
| earthquake | False | 0.999990 |
| earthquake | True  | 0.000010 |
| radio      | False | 0.999990 |
| radio      | True  | 0.000010 |
+------------+-------+----------+
+------------+-------+----------+
| Node       | Value | Marginal |
+------------+-------+----------+
| alarm      | False | 0.000000 |
| alarm*     | True* | 1.000000 |
| burglary   | False | 0.976425 |
| burglary   | True  | 0.023575 |
| earthquake | False | 0.000000 |
| earthquake | True  | 1.000000 |
| radio      | False | 0.000000 |
| radio*     | True* | 1.000000 |
+------------+-------+----------+
+------------+-------+----------+
| Node       | Value | Marginal |
+------------+-------+----------+
| alarm      | False | 0.000000 |
| alarm*     | True* | 1.000000 |
| burglary   |

In [3]:
def p(b=[0,1],e=[0,1],a=[0,1],r=[0,1]):
    if isinstance(b, int): b = [b]
    if isinstance(e, int): e = [e]
    if isinstance(a, int): a = [a]
    if isinstance(r, int): r = [r]
   
    # variable syntax: p_...x := p(x|...)
    p_b = [0.99, 0.01]
    p_e = [1.-10e-6, 10e-6]
    p_er = [[1.0, 0.0], [0.0, 1.0]]
    p_bea = [[[0.999, 0.001], [0.59, 0.41]], [[0.05, 0.95], [0.02, 0.98]]]
 
    p_sum = 0.0
    for bi in b:
        for ei in e:
            for ai in a:
                for ri in r:
                    p_sum += p_b[bi] * p_e[ei] * p_er[ei][ri] * p_bea[bi][ei][ai]

    return p_sum

In [142]:
pb_ar = p(b=1,a=1,r=1) / p(a=1,r=1)
pb_a = p(b=1,a=1) / p(a=1)
print "p(B=1|A=1,R=1) = ", pb_ar
print "p(B=1|A=1) =     ", pb_a

p(B=1|A=1,R=1) =  0.0235746932884
p(B=1|A=1) =      0.905274998587


## (a)(b)

<img src="12-1-ab.jpg" />
<img src="12-1-b-rest.jpg" />

define potentials:

In [5]:
def f(b=[0,1],e=[0,1],a=[0,1]):
    if isinstance(b, int): b = [b]
    if isinstance(e, int): e = [e]
    if isinstance(a, int): a = [a]
   
    # variable syntax: p_...x := p(x|...)
    p_b = [0.99, 0.01]
    p_e = [1.-10e-6, 10e-6]
    p_bea = [[[0.999, 0.001], [0.59, 0.41]], [[0.05, 0.95], [0.02, 0.98]]]
 
    p_sum = 0.0
    for bi in b:
        for ei in e:
            for ai in a:
                p_sum += p_b[bi] * p_e[ei] * p_bea[bi][ei][ai]

    return p_sum

def g(e=[0,1],r=[0,1]):
    if isinstance(e, int): e = [e]
    if isinstance(r, int): r = [r]
   
    # variable syntax: p_...x := p(x|...)
    p_e = [1.-10e-6, 10e-6]
    p_er = [[1.0, 0.0], [0.0, 1.0]]
    
    p_sum = 0.0
    for ei in e:
        for ri in r:
            p_sum += p_e[ei] * p_er[ei][ri]

    return p_sum

def h(e):
    if isinstance(e, int): e = [e]
   
    p_e = [1.-10e-6, 10e-6]
 
    p_sum = 0.0
    for ei in e:
        p_sum += p_e[ei]

    return p_sum

introduce evidence by indicator functions (here: identity-functions `E(ai)=ai`, `E(ri)=ri`):

In [6]:
def f_(b=[0,1],e=[0,1],a=[0,1]):
    if isinstance(b, int): b = [b]
    if isinstance(e, int): e = [e]
    if isinstance(a, int): a = [a]
   
    p_sum = 0.0
    for bi in b:
        for ei in e:
            for ai in a:
                p_sum += f(b=bi,e=ei,a=ai) * ai

    return p_sum

def g_(e=[0,1],r=[0,1]):
    if isinstance(e, int): e = [e]
    if isinstance(r, int): r = [r]
   
    p_sum = 0.0
    for ei in e:
        for ri in r:
            p_sum += g(e=ei,r=ri) * ri

    return p_sum

def h_(e=[0,1]): return h(e)

In [38]:
print "\nP(b=1|a=1)"
print f(b=1,a=1)/f(a=1)  # original function
print f_(b=1)/f_(a=1)  # with indicator function (already incorporates evidence)


P(b=1|a=1)
0.905274998587
0.905274998587


Calculation with functions:

In [113]:
# junction tree:
# (bea) -- [e] -- (er)
#   f       h      g

def test_be(f,g):
    print "\ntest1: computing conditional marginals"
    T = np.zeros((2,2))
    for e in [0,1]:
        for b in [0,1]:
            T[e,b] = f(b=b,e=e) * g(e=e) / h(e=e)
    
    # normalize!
    T = T/T.sum()

    print("e p(..b=0..) p(..b=1..)")
    for e in [0,1]:
        print("%i %.8f %.8f" % (e, T[e,0], T[e,1]))
    return

def test_bea(f,g):
    print "\ntest1: computing conditional marginals"
    T = np.zeros((2,2,2))
    for e in [0,1]:
        for a in [0,1]:
            for b in [0,1]:
                T[e,a,b] = f(b=b,e=e,a=a) * g(e=e) / h(e=e)
    
    # normalize!
    T = T/T.sum()

    print("ea p(..b=0..) p(..b=1..)")
    for e in [0,1]:
        for a in [0,1]:
            print("%i%i %.8f %.8f" % (e,a, T[e,a,0], T[e,a,1]))
    return


# message1 <--
h1 = lambda e: g_(e,1)  # no sum, cause r=1 is observed
f1 = lambda b,e: f_(b,e,1) * h1(e) / h_(e)
test_be(f1,g)

# message2 -->
h2 = lambda e: sum([f1(b,e) for b in [0,1]])
# returns 0 if e==0 to avoid division-by-zero error.
# this is valid, because g_(0,anything) == h2(0) == 0.0  (of course one true 0 in a product would suffice)
g2 = lambda e: 0.0 if not e else h2(e)/h1(e) * g_(e,1)
test_be(f1,g2)

print [g_(e,0) for e in S]

# separator is now h2 (don't do this, your pc will crash)
# h_ = h2


test1: computing conditional marginals
e p(..b=0..) p(..b=1..)
0 0.00000000 0.00000000
1 0.97642531 0.02357469

test1: computing conditional marginals
e p(..b=0..) p(..b=1..)
0 0.00000000 0.00000000
1 0.97642531 0.02357469
[0.0, 0.0]


Calculation with tables (better):

In [148]:
S = [0,1]

print "\nh1(e) = g(e,r=1)"
H1 = [g(e,r=1) for e in S]
print H1

print "\nf1(b,e=1,a) = h1(e=1)/h_(e=1) * f_(b,e=1,a)"
# like in the cell above, skip e==0 to avoid division by zero (the result would have been 0 anyway)
F1 = [[h1(e)/h_(e) * f_(b,e,a) for b in S] for e in [1] for a in S]
print F1

print "\nh2(e) = h1(e)/h_(e) * sum_b[ f(b,e,a=1) ]"
H2 = [(h1(e)/h_(e))*sum([f(b,e,1) for b in S]) for e in S]
print H2

print "\ng2(e=1,r) = h2(e)/h1(e) * g_(e,r)"
# like in the cell above, skip e==0 to avoid division by zero (the result would have been 0 anyway)
G2 = [[(h2(e)/h1(e)) * g_(e,r) for e in [1]] for r in S]
print G2


def prob():
    T = np.zeros(2)
    r = 1
    a = 1
    e = 1  # we haven't observed this, but we can still set this to 1, since T[b] is 0 otherwise (since r=1 <=> e=1)
    for b in [0,1]:
#         T[b] = F1[a][b] * G2[r][0] / H2[e]
        T[b] = F1[a][b] * G2[r][0]
    T = T/T.sum()
    return T

print "\nP(B=1|A=1,R=1)"
print prob()[1]

print "\nP(B=1|A=1)"
print p(b=1,a=1) / p(a=1)


h1(e) = g(e,r=1)
[0.0, 1e-05]

f1(b,e=1,a) = h1(e=1)/h_(e=1) * f_(b,e=1,a)
[[0.0, 0.0], [4.059e-06, 9.8e-08]]

h2(e) = h1(e)/h_(e) * sum_b[ f(b,e,a=1) ]
[0.0, 4.156999999999999e-06]

g2(e=1,r) = h2(e)/h1(e) * g_(e,r)
[[0.0], [4.156999999999999e-06]]

P(B=1|A=1,R=1)
0.0235746932884

P(B=1|A=1)
0.905274998587


## (d) Effect of R=1
Since radio and earthquake are tied so strongly, hearing about it on the radio makes the earthquake much more likely to be the cause of the alarm.

Mathematically, A and R are conditionally independent given E:
    
    P(A|E) = P(A|E,R)

(having observed E, additionally observing R gives us nothing)

Since we haven't observed E though, A and R **are dependent**:

    P(A) != P(A|R)
    
This in turn influences the probability that a burglary caused the alarm.

In [164]:
print [p(a=a) for a in S], "   p(a) ,      a in [0,1]"
print [p(a=a,r=1) / p(r=1) for a in S], "  p(a|r=1) ,  a in [0,1]"

[0.9895059479, 0.0104940521]    p(a) ,      a in [0,1]
[0.5843, 0.41569999999999996]   p(a|r=1) ,  a in [0,1]


---
## (all the important stuff is above, only experiments past here)

In [122]:
# you can use the modified functions to compute the conditionial marginals
def test_bear(f,g):
    T = np.zeros((2,2,2,2))
    for e in [0,1]:
        for a in [0,1]:
            for r in [0,1]:
                for b in [0,1]:
                    T[e,a,r,b] = f(b=b,e=e,a=a) * g(e=e,r=r)
    
    # normalize!
    T = T/T.sum()

    print("ear p(..b=0..) p(..b=1..)")
    for e in [0,1]:
        for a in [0,1]:
            for r in [0,1]:
                print("%i%i%i %.8f %.8f" % (e,a,r, T[e,a,r,0], T[e,a,r,1]))
    return
test_bear(f_,g_)

# but you can't separate the product in a sum if both factors depend on the same variable (e)
def test_sum():
    fsum = sum([f_(b=1,e=e,a=1) for e in [0,1]])
    gsum = sum([g_(e=e,r=1) for e in [0,1]])
    
    T = np.array([fsum, gsum])
    T = T/T.sum()
    print T
    return
test_sum()

ear p(..b=0..) p(..b=1..)
000 0.00000000 0.00000000
001 0.00000000 0.00000000
010 0.00000000 0.00000000
011 0.00000000 0.00000000
100 0.00000000 0.00000000
101 0.00000000 0.00000000
110 0.00000000 0.00000000
111 0.97642531 0.02357469
[ 0.99894848  0.00105152]


# 12.2

## a)
- Bayesian learning belongs to the class of generative models (opposite: discriminative models), where the output prediction $ P(y \vert x) $ is calculated from the marginal pdf $ P(x\vert y) $ and the conditional pdf $ P(x\vert y) $ by applying Bayes' rule
- For Bayesian learning, the inputs $x^{(1)},...,x^{(n)}$ are used to find reasonable model parameters $\theta$. Before using training data to perform the learning algorithm, a prior distribution $P(\theta)$ needs to be defined, by using initial beliefs about the model parameters. When working with MLPs, the model parameters are represented by the weights and biases of the single perceptrons.
- To capture the impact of observations on the model parameters, a likelihood function is introduced: 
$$ L( \theta)=L( \theta | x^{(1)},...,x^{(n)}) \propto P(x^{(1)},...,x^{(n)}| \theta) $$
- After the observation of the input, the pdf $P(\theta | x^{(1)},...,x^{(n)})$ will be updated, now called posterior function by using 
- The predictive distribution is the Bayesian inference and allows us to predict the value of an unknown input $x^{(n+1)}$ given privious inputs by eleminating the model parameters of known distributions: $$ P(x^{(n+1)}|x^{(1)},...,x^{(n)}) \;=\; \int P(x^{(n+1)}| \theta) P( \theta | x^{(1)},...,x^{(n)}) d \theta $$



## b)
- The concept, that model complexity should be limited to avoid an overfitted model doesn't apply to Bayesian learning in a strong matter. This leads to the rule that complexity should only be limited by computational restrictions.
- Bayesian learning is not a method to find scientific explanations or estimate real parameter (nonparametric, since used parameters like weights have no physical meaning for the problem itself) and it's complexity can not simply be defined by the amount of parameters what makes it very different from other approaches.
- When training the model, the weight decay must be well adjusted to avoid both overfitting and underfitting. This is difficult because there is no obvious relationship between the weights and the actual problem.
- Underfitting / overfitting is difficult to control, caused by several reasons. For example the prior distribution needs to be choosen arbitrary and in many cases, choosing them by random delivers resonable results. The same applies to the weight penalty. These problems can partially be reduced by using cross validation. It's also common to work with rules of thumb.
- MLPs are hierarchical what gives them one more degree of freedom and makes them more flexible. 




## c)

- SRM is a deterministic approach for finding a model with a good balance between complexity and the success on test data. Models of the same function class but with different complexities ( possibly represented by VC-dimension or the degree of used functions) are sortet according to their complexity, which might cause overfitting when too large and the empirical (training) error, which tends towards zero for overfitted models. 
- Each model class is tested for the best parameter set (empirical risk minimisation) in order to find the best compromise of empirical error and complexity